In [20]:
# PS4 - CE264

# importing the requried libraries
from collections import OrderedDict    # For recording the model specification 

import pandas as pd                    # For file input/output
import numpy as np                     # For vectorized math operations

import pylogit as pl                   # For MNL model estimation and
                                       # conversion from wide to long format

# reading the data file 
data_raw  = pd.read_csv("BayPass Survey Data_4.24.csv",sep=",")
pd.set_option('display.max_columns', None)
data_tod = pd.read_csv('Dev_TOD.csv')
data_tod.rename(columns ={'ZIP_CODE': 'Zipcode'}, inplace = True)
data_tod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320 entries, 0 to 319
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         320 non-null    int64  
 1   Zipcode            320 non-null    int64  
 2   Percent_Developed  310 non-null    float64
 3   Percent_TOD        215 non-null    float64
dtypes: float64(2), int64(2)
memory usage: 10.1 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Columns: 160 entries, StartDate to F-6-6-3
dtypes: object(160)
memory usage: 187.6+ KB


## Cleaning Data

In [3]:
#Cleaning Survey Data
# reading the data file 
#data_raw  = pd.read_csv("BayPass Survey Data_4.16.csv",sep=",")

data_clean = data_raw.drop([0,1])
data_clean.reset_index(drop= True, inplace = True)
data_clean.sort_values(by = 'StartDate', inplace = True)
display(data_raw)
data_clean = data_clean.drop(columns = data_raw.iloc[:,0:17])

data_clean = data_clean.rename(columns = {'Q2':'Campus Afflication', 'Q3':'Gender', 'Q3_5_TEXT': 'Gender (Other)',
                                          'Q4':'Race/Ethnicity', 'Q4_9_TEXT':'Race/Ethnicity (Other)', 
                                          'Q5':'Employment', 'Q6':'Zipcode', 'Q7':'Modes', 'Q7_12_TEXT':'Modes (Other)',
                                          'Q8': 'BayPass','Q11':'Choice1', 'Q14':'Choice2', 'Q15':'Choice3','Q8':'BayPass','Q16':'Choice4',
                                         'Q17':'Choice5', 'Q18':'Choice6'})
#                                ,
#                                           'F-1-1-1':'S1_TT_Walk','F-1-2-1':'S1_TT_Bike', 'F-1-3-1':'S1_TT_Bus',
#                                          'F-1-4-1': 'S1_TT_Rail', 'F-1-5-1':'S1_TT_Drive','F-1-6-1':'S1_TT_Rideshare',
#                                          'F-1-3-2': 'S1_WT'
                                         
#      
data_clean.columns
data_clean.insert(0,"obs_id",range(1,data_clean.shape[0]+1))
data_obs = data_clean.iloc[:,0:10]
data_obs['BayPass Access'] = data_clean[['BayPass']]

data_q1 = pd.DataFrame({'obs_id': data_clean['obs_id'],'choice': data_clean['Choice1']})
data_q1 = data_q1.join(data_clean.loc[:,'F-1-1':'F-1-6-3'])

data_q2 = pd.DataFrame({'obs_id': data_clean['obs_id'],'choice': data_clean['Choice2']})
data_q2 = data_q2.join(data_clean.loc[:,'F-2-1':'F-2-6-3'])

data_q3 = pd.DataFrame({'obs_id': data_clean['obs_id'],'choice': data_clean['Choice3']})
data_q3 = data_q3.join(data_clean.loc[:,'F-3-1':'F-3-6-3'])

data_q4 = pd.DataFrame({'obs_id': data_clean['obs_id'],'choice': data_clean['Choice4']})
data_q4 = data_q4.join(data_clean.loc[:,'F-4-1':'F-4-6-3'])

data_q5 =  pd.DataFrame({'obs_id': data_clean['obs_id'],'choice': data_clean['Choice5']})
data_q5 = data_q5.join(data_clean.loc[:,'F-5-1':'F-5-6-3'])

data_q6 = pd.DataFrame({'obs_id': data_clean['obs_id'],'choice': data_clean['Choice6']})
data_q6 = data_q6.join(data_clean.loc[:,'F-6-1':'F-6-6-3'])

,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,RecipientFirstName,RecipientEmail,ExternalReference,LocationLatitude,LocationLongitude,DistributionChannel,UserLanguage,Introduction,Q2,Q3,Q3_5_TEXT,Q4,Q4_9_TEXT,Q5,Q6,Q7,Q7_12_TEXT,Q11,Q14,Q15,Q8,Q16,Q17,Q18,F-1-1,F-1-1-1,F-1-2,F-1-1-2,F-1-3,F-1-1-3,F-1-2-1,F-1-2-2,F-1-2-3,F-1-3-1,F-1-3-2,F-1-3-3,F-1-4-1,F-1-4-2,F-1-4-3,F-1-5-1,F-1-5-2,F-1-5-3,F-1-6-1,F-1-6-2,F-1-6-3,F-2-1,F-2-1-1,F-2-2,F-2-1-2,F-2-3,F-2-1-3,F-2-2-1,F-2-2-2,F-2-2-3,F-2-3-1,F-2-3-2,F-2-3-3,F-2-4-1,F-2-4-2,F-2-4-3,F-2-5-1,F-2-5-2,F-2-5-3,F-2-6-1,F-2-6-2,F-2-6-3,F-3-1,F-3-1-1,F-3-2,F-3-1-2,F-3-3,F-3-1-3,F-3-2-1,F-3-2-2,F-3-2-3,F-3-3-1,F-3-3-2,F-3-3-3,F-3-4-1,F-3-4-2,F-3-4-3,F-3-5-1,F-3-5-2,F-3-5-3,F-3-6-1,F-3-6-2,F-3-6-3,F-4-1,F-4-1-1,F-4-2,F-4-1-2,F-4-3,F-4-1-3,F-4-2-1,F-4-2-2,F-4-2-3,F-4-3-1,F-4-3-2,F-4-3-3,F-4-4-1,F-4-4-2,F-4-4-3,F-4-5-1,F-4-5-2,F-4-5-3,F-4-6-1,F-4-6-2,F-4-6-3,F-5-1,F-5-1-1,F-5-2,F-5-1-2,F-5-3,F-5-1-3,F-5-2-1,F-5-2-2,F-5-2-3,F-5-3-1,F-5-3-2,F-5-3-3,F-5-4-1,F-5-4-2,F-5-4-3,F-5-5-1,F-5-5-2,F-5-5-3,F-5-6-1,F-5-6-2,F-5-6-3,F-6-1,F-6-1-1,F-6-2,F-6-1-2,F-6-3,F-6-1-3,F-6-2-1,F-6-2-2,F-6-2-3,F-6-3-1,F-6-3-2,F-6-3-3,F-6-4-1,F-6-4-2,F-6-4-3,F-6-5-1,F-6-5-2,F-6-5-3,F-6-6-1,F-6-6-2,F-6-6-3
0,Start Date,End Date,Response Type,IP Address,Progress,Duration (in seconds),Finished,Recorded Date,Response ID,Recipient Last Name,Recipient First Name,Recipient Email,External Data Reference,Location Latitude,Location Longitude,Distribution Channel,User Language,This survey is designed by UC Berkeley student...,What is your UC Berkeley campus affiliation?,Gender: How do you identify? Select all that a...,Gender: How do you identify? Select all that a...,Race/Ethnicity: How do you identify? Select al...,Race/Ethnicity: How do you identify? Select al...,Are you currently employed? Select all that ap...,What is the zipcode of your Bay Area residence?,What modes of transportation are you likely to...,What modes of transportation are you likely to...,Scenario 1\n\n\n\nWhich mode of transportation...,Scenario 2\n\n\n\nWhich mode of transportation...,Scenario 3\n\n\n\nWhich mode of transportation...,BayPass is an all-inclusive multi-agency trans...,Scenario 4\n\n\n\nWhich mode of transportation...,Scenario 5\n\n\n\nWhich mode of transportation...,Scenario 6\n\n\n\nWhich mode of transportation...,F-1-1,F-1-1-1,F-1-2,F-1-1-2,F-1-3,F-1-1-3,F-1-2-1,F-1-2-2,F-1-2-3,F-1-3-1,F-1-3-2,F-1-3-3,F-1-4-1,F-1-4-2,F-1-4-3,F-1-5-1,F-1-5-2,F-1-5-3,F-1-6-1,F-1-6-2,F-1-6-3,F-2-1,F-2-1-1,F-2-2,F-2-1-2,F-2-3,F-2-1-3,F-2-2-1,F-2-2-2,F-2-2-3,F-2-3-1,F-2-3-2,F-2-3-3,F-2-4-1,F-2-4-2,F-2-4-3,F-2-5-1,F-2-5-2,F-2-5-3,F-2-6-1,F-2-6-2,F-2-6-3,F-3-1,F-3-1-1,F-3-2,F-3-1-2,F-3-3,F-3-1-3,F-3-2-1,F-3-2-2,F-3-2-3,F-3-3-1,F-3-3-2,F-3-3-3,F-3-4-1,F-3-4-2,F-3-4-3,F-3-5-1,F-3-5-2,F-3-5-3,F-3-6-1,F-3-6-2,F-3-6-3,F-4-1,F-4-1-1,F-4-2,F-4-1-2,F-4-3,F-4-1-3,F-4-2-1,F-4-2-2,F-4-2-3,F-4-3-1,F-4-3-2,F-4-3-3,F-4-4-1,F-4-4-2,F-4-4-3,F-4-5-1,F-4-5-2,F-4-5-3,F-4-6-1,F-4-6-2,F-4-6-3,F-5-1,F-5-1-1,F-5-2,F-5-1-2,F-5-3,F-5-1-3,F-5-2-1,F-5-2-2,F-5-2-3,F-5-3-1,F-5-3-2,F-5-3-3,F-5-4-1,F-5-4-2,F-5-4-3,F-5-5-1,F-5-5-2,F-5-5-3,F-5-6-1,F-5-6-2,F-5-6-3,F-6-1,F-6-1-1,F-6-2,F-6-1-2,F-6-3,F-6-1-3,F-6-2-1,F-6-2-2,F-6-2-3,F-6-3-1,F-6-3-2,F-6-3-3,F-6-4-1,F-6-4-2,F-6-4-3,F-6-5-1,F-6-5-2,F-6-5-3,F-6-6-1,F-6-6-2,F-6-6-3
1,"{""ImportId"":""startDate"",""timeZone"":""America/De...","{""ImportId"":""endDate"",""timeZone"":""America/Denv...","{""ImportId"":""status""}","{""ImportId"":""ipAddress""}","{""ImportId"":""progress""}","{""ImportId"":""duration""}","{""ImportId"":""finished""}","{""ImportId"":""recordedDate"",""timeZone"":""America...","{""ImportId"":""_recordId""}","{""ImportId"":""recipientLastName""}","{""ImportId"":""recipientFirstName""}","{""ImportId"":""recipientEmail""}","{""ImportId"":""externalDataReference""}","{""ImportId"":""locationLatitude""}","{""ImportId"":""locationLongitude""}","{""ImportId"":""distribut

In [4]:
# Looping through each of the question options, dropping unnecessarily values, including fields not included, and renaming column

for df in [data_q1, data_q2, data_q3, data_q4, data_q5, data_q6]:
    df.drop(df.columns[[2,4,6]],axis = 1, inplace = True)
    df.rename(columns = {df.columns[2]:'tt_walk', df.columns[3]:'wt_walk', df.columns[4]:'cost_walk',
                         df.columns[5]:'tt_bike', df.columns[6]:'wt_bike', df.columns[7]:'cost_bike',
                         df.columns[8]: 'tt_bus', df.columns[9]: 'wt_bus', df.columns[10]:'cost_bus',
                         df.columns[11]: 'tt_rail', df.columns[12]: 'wt_rail', df.columns[13]:'cost_rail',
                         df.columns[14]:'tt_drive', df.columns[15]: 'wt_drive', df.columns[16]:'cost_drive',
                         df.columns[17]: 'tt_rideshare', df.columns[18]: 'wt_rideshare', df.columns[19]:'cost_rideshare'
                        }, inplace = True)
    df[['wt_walk', 'wt_bike', 'cost_walk', 'cost_bike']] = 0 
    df['BayPass']= 0
    
for df in [data_q4, data_q5, data_q6]:
    df[['cost_bus', 'cost_rail']] = 0
    df['BayPass'] = 1


In [5]:
display(data_obs)
# for df in [data_q1, data_q2, data_q3, data_q4, data_q5, data_q6]:
# df = data_obs.merge(df, how = 'inner', on = ['obs_id'], inplace =True)



,obs_id,Introduction,Campus Afflication,Gender,Gender (Other),Race/Ethnicity,Race/Ethnicity (Other),Employment,Zipcode,Modes,BayPass Access
0,1,"Yes, I consent",Graduate student,Female,NaN,Asian,NaN,"Yes, employed off campus",94704,"AC Transit,BART,Walk,Personal bike",No
1,2,"Yes, I consent",Graduate student,Male,NaN,"White,Native American or Indigenous,Hispanic o...",NaN,"Yes, employed on campus as a student",94703,"AC Transit,BART,Walk,Shared micromobility (ex:...",No
4,3,"Yes, I consent",Graduate student,Male,NaN,"White,Asian",NaN,"Yes, employed off campus,Yes, employed on camp...",94704,"AC Transit,BART,Walk,Drive alone,Rideshare (ex...",No
2,4,"Yes, I consent",Graduate student,Female,NaN,White,NaN,"Yes, employed off campus",94703,"AC Transit,BART,Walk,Personal bike,Drive alone",No
3,5,"Yes, I consent",Alumnus,Female,NaN,Asian,NaN,No,NaN,"AC Transit,BART,Walk,Drive alone,Carpool,Rides...",No
...,...,...,...,...,...,...,...,...,...,...,...
138,144,"Yes, I consent",Graduate student,Male,NaN,Hispanic or Latinx,NaN,No,94704,"AC Transit,BART,Walk,Personal bike",No
144,145,"Yes, I consent",Undergraduate student,Female,NaN,White,NaN,No,NaN,"BART,Walk,Personal scooter, e-scooter, skatebo...",Yes
145,146,"Yes, I consent",Undergraduate student,Female,NaN,Prefer not to say,NaN,"Yes, employed off campus",94549,"Bear Transit,AC Transit,BART,Walk,Drive alone,...",Not sure
146,147,"Yes, I consent",Undergraduate student,Female,NaN,White,NaN,"Yes, employed off campus,Yes, employed on camp...",94609,"AC Transit,BART,Walk,Personal bike",Yes


In [23]:
dflist = [data_q1, data_q2, data_q3, data_q4, data_q5, data_q6]
data = pd.concat(dflist)
data = pd.merge(data_obs, data, on = 'obs_id', how = 'right')
data.sort_values(by = 'obs_id', inplace = True)
data.reset_index(drop= True, inplace = True)

#display(data)

In [28]:
## Recode Data and Clean
data.dropna(subset = ["choice"], inplace = True)
data.fillna("Did Not Response", inplace = True)
data['Zipcode'].replace('Did Not Response', 0, inplace = True)
data['Zipcode'] = data['Zipcode'].astype(int)

data = data.merge(data_tod, on = 'Zipcode', how = 'left')


alt = ['Walk', 'Bike or Scooter', 'Bus', 'Rail (BART, Caltrain)','Drive', 'Rideshare']
dummy_var = [1,2,3,4,5,6]
data['choice'] = data['choice'].replace(alt, dummy_var)

student_alt = ['Undergraduate student', 'Graduate student','Postdoc', 'Faculty member', 'Non-faculty university employee', 'Retired or emeritus status', 'Alumnus', 'I am not affiliated with UC Berkeley']
student_dummy = [1,1,0,0,0,0,0,0]
data['student_dummy'] = data['Campus Afflication'].replace(student_alt, student_dummy)

employed_alt = ['Yes, employed off campus', 'Yes, employed on campus as a student', 'Yes, employed on campus as faculty/staff','No']
employed_dum = [1,1,1,0]
data['employed_dummy'] = data['Employment']
data['employed_dummy'].replace(employed_alt, employed_dum, inplace = True)



display(data)



#Introduce baypass as a dummy variable - value of free is not linear
# Cost Parameter and another parameter for free (Significant - non linearity)
# Forecasting - how do we figure out weights - need a different situation
# Take sample dataset and infer travel time and cost for different alternatives - use this as our base
# Estimate mode share w/o baypass, Estimate mode share w/ baypass 
# One model - base will zero out baypass - BayPass Dummy (Base Case: Zero this out)
# If everyone has baypass how much more transit, by whom, zipcode
# segment survey to transit oriented and less transit oriented locations
# Weight sample - don't worry too much (not a random sample - select one variable and weight it (zip, gender, campus afflication, race/ethncity))
# student vs non-student (Incorporate dummy for non-student)
# 

,obs_id,Introduction,Campus Afflication,Gender,Gender (Other),Race/Ethnicity,Race/Ethnicity (Other),Employment,Zipcode,Modes,BayPass Access,choice,tt_walk,wt_walk,cost_walk,tt_bike,wt_bike,cost_bike,tt_bus,wt_bus,cost_bus,tt_rail,wt_rail,cost_rail,tt_drive,wt_drive,cost_drive,tt_rideshare,wt_rideshare,cost_rideshare,BayPass,Unnamed: 0_x,Percent_Developed_x,Percent_TOD_x,student_dummy,employed_dummy,Unnamed: 0_y,Percent_Developed_y,Percent_TOD_y
0,1,"Yes, I consent",Graduate student,Female,Did Not Response,Asian,Did Not Response,"Yes, employed off campus",94704,"AC Transit,BART,Walk,Personal bike",No,1,10,0,0,40,0,0,30,10,10,40,5,1,5,20,4,45,0,7,0,168.0,93.184868,98.486878,1,1,168.0,93.184868,98.486878
1,1,"Yes, I consent",Graduate student,Female,Did Not Response,Asian,Did Not Response,"Yes, employed off campus",94704,"AC Transit,BART,Walk,Personal bike",No,2,45,0,0,15,0,0,10,15,1,10,0,10,40,0,8,20,10,8,0,168.0,93.184868,98.486878,1,1,168.0,93.184868,98.486878
2,1,"Yes, I consent",Graduate student,Female,Did Not Response,Asian,Did Not Response,"Yes, employed off campus",94704,"AC Transit,BART,Walk,Personal bike",No,4,35,0,0,30,0,0,40,0,0,20,10,0,30,5,7,40,0,8,1,168.0,93.184868,98.486878,1,1,168.0,93.184868,98.486878
3,1,"Yes, I consent",Graduate student,Female,Did Not Response,Asian,Did Not Response,"Yes, employed off campus",94704,"AC Transit,BART,Walk,Personal bike",No,2,25,0,0,5,0,0,30,10,0,15,5,0,15,10,3,40,0,4,1,168.0,93.184868,98.486878,1,1,168.0,93.184868,98.486878
4,1,"Yes, I consent",Graduate student,Female,Did Not Response,Asian,Did Not Response,"Yes, employed off campus",94704,"AC Transit,BART,Walk,Personal bike",No,1,20,0,0,25,0,0,15,15,0,35,10,0,35,15,4,35,5,4,1,168.0,93.184868,98.486878,1,1,168.0,93.184868,98.486878
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
758,148,"Yes, I consent",Undergraduate student,Female,Did Not Response,White,Did Not Response,"Yes, employed on campus as a student",94704,"AC Transit,BART,Walk,Personal bike,Carpool",Yes,3,55,0,0,30,0,0,12,15,0,5,10,0,25,0,6,25,20,6,1,168.0,93.184868,98.486878,1,1,168.0,93.184868,98.486878
759,148,"Yes, I consent",Undergraduate student,Female,Did Not Response,White,Did Not Response,"Yes, employed on campus as a student",94704,"AC Transit,BART,Walk,Personal bike,Carpool",Yes,3,45,0,0,30,0,0,12,15,2,15,7,7,12,7,8,25,10,15,0,168.0,93.184868,98.486878,1,1,168.0,93.184868,98.486878
760,148,"Yes, I consent",Undergraduate student,Female,Did Not Response,White,Did Not Response,"Yes, employed on campus as a student",94704,"AC Transit,BART,Walk,Personal bike,Carpool",Yes,2,50,0,0,15,0,0,12,0,1,20,15,6,5,12,1,20,15,6,0,168.0,93.184868,98.486878,1,1,168.0,93.184868,98.486878
761,148,"Yes, I consent",Undergraduate student,Female,Did Not Response,White,Did Not Response,"Yes, employed on campus as a student",94704,"AC Transit,BART,Walk,Personal bike,Carpool",Yes,3,55,0,0,30,0,0,15,0,0,25,10,0,25,10,3,15,7,7,1,168.0,93.184868,98.486878,1,1,168.0,93.184868,98.486878


In [30]:
#Export Data
data.to_csv('Cleaned BayPass Survey_4.24.csv')